# Projeto de Web Scraping para Scouting de Times de Valorant: Análise de Pickrate de Mapas e Agentes

**Feito por:** Guilherme Lunetta

O crescimento do Valorant como um jogo competitivo de tiro em primeira pessoa tem impulsionado a necessidade de estratégias avançadas para equipes profissionais. Uma das informações mais cruciais para o sucesso de um time é o pickrate (taxa de escolha) de mapas e agentes de cada jogador. Para coletar esses dados de forma eficiente, um projeto de web scraping foi desenvolvido, visando extrair informações relevantes das fontes online disponíveis.

O scouting no contexto dos e-sports refere-se ao processo de coletar informações detalhadas e analisar o desempenho de equipes e jogadores profissionais em um determinado jogo eletrônico. Assim como em esportes tradicionais, o scouting nos e-sports é fundamental para entender as estratégias, habilidades e pontos fortes de cada equipe e jogador, a fim de tomar decisões informadas durante competições.

Ao coletar e analisar essas informações, os scouts podem identificar as fortalezas e fraquezas de uma equipe, bem como os pontos fortes e fracos individuais de cada jogador. Esses dados são essenciais para a tomada de decisões estratégicas, como a seleção de mapas e agentes, a adaptação às estratégias dos oponentes e o desenvolvimento de táticas personalizadas.

Em resumo, o scouting em e-sports é um processo de coleta e análise de informações sobre equipes e jogadores profissionais, visando entender suas estratégias, habilidades e pontos fortes. Essa prática é fundamental para aprimorar a tomada de decisões estratégicas e alcançar o sucesso nas competições de jogos eletrônicos.

In [27]:
# Importando bibliotecas necessárias para o projeto

import requests
import time
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import os
from IPython.display import display, Markdown

In [7]:
# Funções utilizadas ao longo do projeto

def extract_bans(bans):
    if len(bans) == 2:
        bans = bans[1].text.strip()
    else:
        bans = bans[0].text.strip()
        
    res_dic = {
        "Bans": [],
        "Picks": [],
    }
    
    teams_ban = bans.split(";")
    for ban in teams_ban:
        content = ban.strip().split(" ")
        if content[0] == "C9" or content[0] == "LEV" or content[0] == "NRG":
            if content[1] == "ban":
                res_dic["Bans"].append(content[2])
            else:
                res_dic["Picks"].append(content[2])
    
    return res_dic

def extract_maps(bans):
    if len(bans) == 2:
        bans = bans[1].text.strip()
    else:
        bans = bans[0].text.strip()
        
    maps = []
    
    teams_bans = bans.split(";")
    for ban in teams_bans:
        content = ban.strip().split(" ")
        if content[1] == "pick":
            maps.append(content[2])
            
        if len(content) == 2:
            maps.append(content[0])
            
    return maps

In [8]:
# URL do site usado no webscrapping

base_url = "https://www.vlr.gg"

## Cloud9, os novatos endiabrados

O time da Cloud9 no Valorant é uma equipe profissional que compete na região das Américas. No início do semestre, o time teve uma surpresa com a saída do jogador "yay", o que exigiu uma reestruturação na formação da equipe. No entanto, ao longo do campeonato VCT Americas, a Cloud9 mostrou um crescimento notável e se estabeleceu como uma grande potência no cenário.

Após a mudança em sua formação, a Cloud9 demonstrou resiliência e habilidade para se adaptar rapidamente às adversidades. Com uma combinação de jogadores experientes e talentosos, a equipe começou a construir uma identidade forte e a desenvolver estratégias eficazes.

Ao longo do campeonato VCT Americas, a Cloud9 surpreendeu a todos com seu desempenho consistente e seu jogo agressivo. Com táticas bem elaboradas, coordenação eficiente e individualidades brilhantes, o time conquistou vitórias impressionantes contra adversários considerados favoritos.

In [9]:
# Começando scrapping dos dados da Cloud9

c9url = "https://www.vlr.gg/team/matches/188/cloud9"

res = requests.get(c9url)

soup = bs(res.text, 'html.parser')

In [10]:
# Escolhendo jogos apenas do VCT Americas (campeonato mais recente)

matches = soup.findAll('a', attrs = {'class': 'wf-card fc-flex m-item'})

match_links = []

for match in matches:
    camp = match.find('div', attrs = {'class': 'text-of'}).text.strip().split(":")[0]
    if camp == "VCT 2023":
    	match_links.append(base_url + match.get('href'))

In [11]:
# Entrando em cada jogo e coletando as informações necessárias

c9_games = {}
for link in match_links: 
    r = requests.get(link)
    soup = bs(r.text, 'html.parser')
    
    date = soup.find('div', {'class': 'moment-tz-convert'}).text.strip()
    home_team = soup.find('div', {'class': 'match-header-link-name mod-1'}).find_all('div')[0].text.strip()
    away_team = soup.find('div', {'class': 'match-header-link-name mod-2'}).find_all('div')[0].text.strip()
    
    is_c9_away = True
    if home_team == "Cloud9":
        opponent = away_team
        is_c9_away = False
    else:
        opponent = home_team
        
    print(f"Raspando informações do jogo: Cloud9 vs {opponent}")
    
    bans = soup.find_all('div', attrs = {'class': 'match-header-note'})
    c9_bans = extract_bans(bans)
    
    if home_team == "Cloud9":
        c9_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[0].text.strip()
        opponent_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[2].text.strip()
    else:
        c9_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[2].text.strip()
        opponent_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[0].text.strip()
    
    games = int(c9_score) + int(opponent_score) 
    first_map_id = int(soup.find('div', {'class': 'vm-stats-gamesnav-item js-map-switch'}).get('data-game-id'))
    
    maps = extract_maps(bans)
    
    maps_info = {}
    
    i = 0
    for gid in range(first_map_id, first_map_id+games):
        map_info = {}
        print(f"Raspando informações do mapa {maps[i]} com id: {gid}")
        
        map_url = f"{link}/?game={gid}&tab=overview"
        r = requests.get(map_url)
        soup = bs(r.text, 'html.parser')
        
        map_name = soup.find('div', attrs = {'class': 'map'}).find_all('span')[0].text.strip()
        
        c9_rounds = soup.find('div', attrs = {'class': 'score mod-win'}).text.strip()
        opponent_rounds = soup.find('div', attrs = {'class': 'score'}).text.strip()
        
        tables = soup.find_all('table', {'class': 'wf-table-inset mod-overview'})
        
        if is_c9_away:
            team = tables[1].find('tbody')
        else:
            team = tables[0].find('tbody')
        
        for tr in team.find_all('tr'):
            player = tr.find('td', attrs = {'class': 'mod-player'}).find('div', attrs = {'class': 'text-of'}).text.strip()
            agent = tr.find('td', attrs = {'class': 'mod-agents'}).find('img').get('title')
            map_info[player] = agent
            
        maps_info[maps[i]] = map_info
        
        i+=1
        time.sleep(1)
        
    c9_games[f"cloud9-{opponent.lower().replace('á', 'a')}"] = maps_info

Raspando informações do jogo: Cloud9 vs NRG Esports
Raspando informações do mapa Pearl com id: 123185
Raspando informações do mapa Lotus com id: 123186
Raspando informações do mapa Haven com id: 123187
Raspando informações do jogo: Cloud9 vs Leviatán
Raspando informações do mapa Haven com id: 123179
Raspando informações do mapa Lotus com id: 123180
Raspando informações do mapa Ascent com id: 123181
Raspando informações do jogo: Cloud9 vs Evil Geniuses
Raspando informações do mapa Fracture com id: 123168
Raspando informações do mapa Bind com id: 123169
Raspando informações do jogo: Cloud9 vs KRÜ Esports
Raspando informações do mapa Lotus com id: 119237
Raspando informações do mapa Ascent com id: 119238
Raspando informações do mapa Split com id: 119239
Raspando informações do jogo: Cloud9 vs Leviatán
Raspando informações do mapa Lotus com id: 119040
Raspando informações do mapa Haven com id: 119041
Raspando informações do jogo: Cloud9 vs MIBR
Raspando informações do mapa Split com id: 11

In [12]:
# Salvando as informações da C9 em um arquivo JSON

with open("cloud9.json", "w") as f:
    json.dump(c9_games, f, indent=2)

## Leviatán, o super time da América Latina

A Leviatan é uma equipe proeminente no cenário competitivo de Valorant. Com jogadores talentosos e uma abordagem estratégica excepcional, a Leviatan se destaca como uma das potências do jogo.

Desde sua formação, a Leviatan tem demonstrado uma notável consistência e habilidade em se adaptar às demandas do jogo. Com uma abordagem agressiva e táticas bem planejadas, a equipe conquistou vitórias impressionantes em várias competições importantes.

O sucesso da Leviatan pode ser atribuído não apenas às habilidades individuais de seus jogadores, mas também à coesão do time como um todo. A comunicação efetiva e o trabalho em equipe são características marcantes da Leviatan, o que lhes permite tomar decisões rápidas e estratégicas durante as partidas.

In [13]:
# Scrapping dos dados da Leviatán

levurl = "https://www.vlr.gg/team/matches/2359/leviat-n/"

res = requests.get(levurl)

soup = bs(res.text, 'html.parser')

In [14]:
# Escolhendo jogos apenas do VCT Americas (campeonato mais recente)

matches = soup.findAll('a', attrs = {'class': 'wf-card fc-flex m-item'})

match_links = []

for match in matches:
    camp = match.find('div', attrs = {'class': 'text-of'}).text.strip().split(":")[0]
    if camp == "VCT 2023":
    	match_links.append(base_url + match.get('href'))

In [15]:
# Entrando em cada jogo e coletando as informações necessárias

lev_games = {}
for link in match_links: 
    r = requests.get(link)
    soup = bs(r.text, 'html.parser')
    
    date = soup.find('div', {'class': 'moment-tz-convert'}).text.strip()
    home_team = soup.find('div', {'class': 'match-header-link-name mod-1'}).find_all('div')[0].text.strip()
    away_team = soup.find('div', {'class': 'match-header-link-name mod-2'}).find_all('div')[0].text.strip()
    
    is_lev_away = True
    if home_team == "Leviatán":
        opponent = away_team
        is_lev_away = False
    else:
        opponent = home_team
        
    print(f"Raspando informações do jogo: Leviatán vs {opponent}")
    
    bans = soup.find_all('div', attrs = {'class': 'match-header-note'})
    lev_bans = extract_bans(bans)
    
    if home_team == "Leviatán":
        lev_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[0].text.strip()
        opponent_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[2].text.strip()
    else:
        lev_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[2].text.strip()
        opponent_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[0].text.strip()
    
    games = int(lev_score) + int(opponent_score) 
    first_map_id = int(soup.find('div', {'class': 'vm-stats-gamesnav-item js-map-switch'}).get('data-game-id'))
    
    maps = extract_maps(bans)
    
    maps_info = {}
    
    i = 0
    for gid in range(first_map_id, first_map_id+games):
        map_info = {}
        print(f"Raspando informações do mapa {maps[i]} com id: {gid}")
        
        map_url = f"{link}/?game={gid}&tab=overview"
        r = requests.get(map_url)
        soup = bs(r.text, 'html.parser')
        
        map_name = soup.find('div', attrs = {'class': 'map'}).find_all('span')[0].text.strip()
        
        lev_rounds = soup.find('div', attrs = {'class': 'score mod-win'}).text.strip()
        opponent_rounds = soup.find('div', attrs = {'class': 'score'}).text.strip()
        
        tables = soup.find_all('table', {'class': 'wf-table-inset mod-overview'})
        
        if is_lev_away:
            team = tables[1].find('tbody')
        else:
            team = tables[0].find('tbody')
        
        for tr in team.find_all('tr'):
            player = tr.find('td', attrs = {'class': 'mod-player'}).find('div', attrs = {'class': 'text-of'}).text.strip()
            agent = tr.find('td', attrs = {'class': 'mod-agents'}).find('img').get('title')
            map_info[player] = agent
            
        maps_info[maps[i]] = map_info
        
        i+=1
        time.sleep(1)
        
    lev_games[f"leviatan-{opponent.lower().replace('ü', 'u')}"] = maps_info

Raspando informações do jogo: Leviatán vs Cloud9
Raspando informações do mapa Haven com id: 123179
Raspando informações do mapa Lotus com id: 123180
Raspando informações do mapa Ascent com id: 123181
Raspando informações do jogo: Leviatán vs FURIA
Raspando informações do mapa Ascent com id: 123159
Raspando informações do mapa Pearl com id: 123160
Raspando informações do mapa Bind com id: 123161
Raspando informações do jogo: Leviatán vs LOUD
Raspando informações do mapa Haven com id: 119240
Raspando informações do mapa Ascent com id: 119241
Raspando informações do jogo: Leviatán vs Cloud9
Raspando informações do mapa Lotus com id: 119040
Raspando informações do mapa Haven com id: 119041
Raspando informações do jogo: Leviatán vs KRÜ Esports
Raspando informações do mapa Haven com id: 119022
Raspando informações do mapa Split com id: 119023
Raspando informações do jogo: Leviatán vs Evil Geniuses
Raspando informações do mapa Split com id: 119007
Raspando informações do mapa Pearl com id: 11

In [16]:
# Salvando as informações da LEV em um arquivo JSON

with open("leviatan.json", "w") as f:
    json.dump(lev_games, f, indent=2)

## NRG, constantes e perigosos

A NRG é uma equipe de destaque no cenário competitivo de Valorant. Com uma formação talentosa e uma abordagem estratégica excepcional, a NRG se estabeleceu como uma das principais potências do jogo.

A força da NRG reside na combinação de habilidades individuais de seus jogadores e na sinergia que eles demonstram como equipe. Cada jogador traz sua própria expertise e estilo de jogo único, o que contribui para uma dinâmica de equipe diversificada e eficiente.

Ao longo das competições, a NRG tem mostrado uma consistência notável, alcançando resultados impressionantes. Seja através de estratégias bem planejadas, controle de mapa habilidoso ou jogadas individuais brilhantes, a equipe é capaz de superar os obstáculos que surgem em seu caminho.

In [17]:
# Scrapping dos dados da NRG

nrgurl = "https://www.vlr.gg/team/matches/1034/nrg-esports/"

res = requests.get(nrgurl)

soup = bs(res.text, 'html.parser')

In [18]:
# Escolhendo jogos apenas do VCT Americas (campeonato mais recente)

matches = soup.findAll('a', attrs = {'class': 'wf-card fc-flex m-item'})

match_links = []

for match in matches:
    camp = match.find('div', attrs = {'class': 'text-of'}).text.strip().split(":")[0]
    if camp == "VCT 2023":
    	match_links.append(base_url + match.get('href'))

In [19]:
# Entrando em cada jogo e coletando informações necessárias

nrg_games = {}
for link in match_links: 
    r = requests.get(link)
    soup = bs(r.text, 'html.parser')
    
    date = soup.find('div', {'class': 'moment-tz-convert'}).text.strip()
    home_team = soup.find('div', {'class': 'match-header-link-name mod-1'}).find_all('div')[0].text.strip()
    away_team = soup.find('div', {'class': 'match-header-link-name mod-2'}).find_all('div')[0].text.strip()
    
    is_nrg_away = True
    if home_team == "NRG Esports":
        opponent = away_team
        is_nrg_away = False
    else:
        opponent = home_team
        
    print(f"Raspando informações do jogo: NRG vs {opponent}")
    
    bans = soup.find_all('div', attrs = {'class': 'match-header-note'})
    nrg_bans = extract_bans(bans)
    
    if home_team == "NRG Esports":
        nrg_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[0].text.strip()
        opponent_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[2].text.strip()
    else:
        nrg_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[2].text.strip()
        opponent_score = soup.find('div', {'class': 'js-spoiler'}).find_all('span')[0].text.strip()
    
    games = int(nrg_score) + int(opponent_score) 
    first_map_id = int(soup.find('div', {'class': 'vm-stats-gamesnav-item js-map-switch'}).get('data-game-id'))
    
    maps = extract_maps(bans)
    
    maps_info = {}
    
    i = 0
    for gid in range(first_map_id, first_map_id+games):
        map_info = {}
        print(f"Raspando informações do mapa {maps[i]} com id: {gid}")
        
        map_url = f"{link}/?game={gid}&tab=overview"
        r = requests.get(map_url)
        soup = bs(r.text, 'html.parser')
        
        map_name = soup.find('div', attrs = {'class': 'map'}).find_all('span')[0].text.strip()
        
        nrg_rounds = soup.find('div', attrs = {'class': 'score mod-win'}).text.strip()
        opponent_rounds = soup.find('div', attrs = {'class': 'score'}).text.strip()
        
        tables = soup.find_all('table', {'class': 'wf-table-inset mod-overview'})
        
        if is_nrg_away:
            team = tables[1].find('tbody')
        else:
            team = tables[0].find('tbody')
        
        for tr in team.find_all('tr'):
            player = tr.find('td', attrs = {'class': 'mod-player'}).find('div', attrs = {'class': 'text-of'}).text.strip()
            agent = tr.find('td', attrs = {'class': 'mod-agents'}).find('img').get('title')
            map_info[player] = agent
            
        maps_info[maps[i]] = map_info
        
        i+=1
        time.sleep(1)
        
    nrg_games[f"leviatan-{opponent.lower().replace('ü', 'u').replace('á', 'a')}"] = maps_info

Raspando informações do jogo: NRG vs Evil Geniuses
Raspando informações do mapa Haven com id: 123188
Raspando informações do mapa Ascent com id: 123189
Raspando informações do mapa Lotus com id: 123190
Raspando informações do mapa Split com id: 123191
Raspando informações do jogo: NRG vs Cloud9
Raspando informações do mapa Pearl com id: 123185
Raspando informações do mapa Lotus com id: 123186
Raspando informações do mapa Haven com id: 123187
Raspando informações do jogo: NRG vs FURIA
Raspando informações do mapa Bind com id: 123182
Raspando informações do mapa Haven com id: 123183
Raspando informações do jogo: NRG vs Evil Geniuses
Raspando informações do mapa Haven com id: 123162
Raspando informações do mapa Split com id: 123163
Raspando informações do mapa Ascent com id: 123164
Raspando informações do jogo: NRG vs Evil Geniuses
Raspando informações do mapa Bind com id: 119234
Raspando informações do mapa Split com id: 119235
Raspando informações do jogo: NRG vs LOUD
Raspando informaçõ

In [21]:
# Salvando as informações da NRG em um arquivo JSON

with open("nrg-esports.json", "w") as f:
    json.dump(nrg_games, f, indent=2)

# Pickrate de Mapas e Agentes

O pickrate de mapas e agentes no Valorant refere-se à frequência com que cada mapa e agente é escolhido durante as partidas. Essa métrica é importante porque fornece insights sobre as preferências e popularidade de escolha dos jogadores.

Ao analisar o pickrate de mapas, é possível identificar quais são os mapas mais populares entre os times, quais são menos escolhidos e entender a dinâmica de jogo em cada um deles. Isso auxilia na preparação estratégica das equipes, permitindo que elas se concentrem em aprimorar suas táticas nos mapas mais jogados e compreendam as particularidades de cada ambiente de jogo.

Já o pickrate de agentes revela quais personagens são mais frequentemente selecionados pelos jogadores. Isso oferece informações valiosas sobre as preferências de cada um e a eficácia de cada agente em diferentes situações. Com base nesses dados, os jogadores e equipes podem ajustar suas estratégias, criar composições de equipe mais equilibradas e adaptar-se ao meta atual do jogo.

## Cloud9 - leaf, runi, xeppaa, jakee e zellsis

In [31]:
# Análise do arquivo JSON da Cloud9

with open("cloud9.json", "r") as f:
    games = json.load(f)
    
map_picks = {}
total_picks = 0

for team, maps in games.items():
    for _ in maps.values():
        total_picks += 1

for game in games.keys():
    for mapa in games[game].keys():
        if map_picks.get(mapa) != None:
            map_picks[mapa] += 1
        else:
            map_picks[mapa] = 1
           
display(Markdown("**Pickrate de mapas:**"))
for mapa, num in map_picks.items():
    pctg = round((num/total_picks)*100, 2)
    print(f"{mapa} -> {pctg}%")

players = {}
for game in games:
    for mapa in games[game].keys():
        for player, agent in games[game][mapa].items():
            if players.get(player) != None:
                if (players[player].get(mapa) != None):
                    players[player][mapa].append(agent)
                else:
                    players[player][mapa] = []
            else:
                players[player] = {}
                players[player][mapa] = [agent]
                
# Stats de cada jogador com cada agente

display(Markdown("**Pickrate de cada player:**"))
for player in players.keys():
    display(Markdown(f"**Player: {player}**"))
    for mapa in players[player].keys():
        print("Map:", mapa)
        for agent in set(players[player][mapa]):
            agent_pctg = players[player][mapa].count(agent)/len(players[player][mapa])
            print("Agent: {} - Pick Rate: {:.2f}%".format(agent, agent_pctg*100))
        print("")
    print("")

**Pickrate de mapas:**

Haven -> 9.52%
Pearl -> 14.29%
Ascent -> 23.81%
Lotus -> 23.81%
Fracture -> 14.29%
Split -> 14.29%


**Pickrate de cada player:**

**Player: leaf**

Map: Haven
Agent: Neon - Pick Rate: 50.00%
Agent: Jett - Pick Rate: 50.00%

Map: Pearl
Agent: Killjoy - Pick Rate: 50.00%
Agent: Skye - Pick Rate: 50.00%

Map: Ascent
Agent: Neon - Pick Rate: 50.00%
Agent: Jett - Pick Rate: 25.00%
Agent: Skye - Pick Rate: 25.00%

Map: Lotus
Agent: Neon - Pick Rate: 75.00%
Agent: Skye - Pick Rate: 25.00%

Map: Fracture
Agent: Neon - Pick Rate: 50.00%
Agent: Jett - Pick Rate: 50.00%

Map: Split
Agent: Skye - Pick Rate: 100.00%




...

Map: Haven
Agent: Breach - Pick Rate: 50.00%
Agent: Skye - Pick Rate: 50.00%

Map: Pearl
Agent: Skye - Pick Rate: 50.00%
Agent: Raze - Pick Rate: 50.00%

Map: Ascent
Agent: Skye - Pick Rate: 75.00%
Agent: Raze - Pick Rate: 25.00%

Map: Lotus
Agent: Skye - Pick Rate: 75.00%
Agent: Raze - Pick Rate: 25.00%

Map: Fracture
Agent: Skye - Pick Rate: 100.00%

Map: Split
Agent: Raze - Pick Rate: 100.00%




**Player: Zellsis**

Map: Haven
Agent: Killjoy - Pick Rate: 100.00%

Map: Pearl
Agent: Killjoy - Pick Rate: 50.00%
Agent: Phoenix - Pick Rate: 50.00%

Map: Ascent
Agent: Killjoy - Pick Rate: 100.00%

Map: Lotus
Agent: Killjoy - Pick Rate: 100.00%

Map: Fracture
Agent: Killjoy - Pick Rate: 100.00%

Map: Split
Agent: Killjoy - Pick Rate: 100.00%




**Player: jakee**

Map: Haven
Agent: Omen - Pick Rate: 100.00%

Map: Pearl
Agent: Astra - Pick Rate: 100.00%

Map: Ascent
Agent: Omen - Pick Rate: 75.00%
Agent: Astra - Pick Rate: 25.00%

Map: Lotus
Agent: Omen - Pick Rate: 75.00%
Agent: Astra - Pick Rate: 25.00%

Map: Fracture
Agent: Omen - Pick Rate: 100.00%

Map: Split
Agent: Astra - Pick Rate: 100.00%




In [33]:
# Análise do arquivo JSON da Leviatan

with open("leviatan.json", "r") as f:
    games = json.load(f)

map_picks = {}
total_picks = 0

for team, maps in games.items():
    for _ in maps.values():
        total_picks += 1

for game in games.keys():
    for mapa in games[game].keys():
        if map_picks.get(mapa) != None:
            map_picks[mapa] += 1
        else:
            map_picks[mapa] = 1
           
display(Markdown("**Pickrate de mapas:**"))
for mapa, num in map_picks.items():
    pctg = round((num/total_picks)*100, 2)
    print(f"{mapa} -> {pctg}%")
    

players = {}
for game in games:
    for mapa in games[game].keys():
        for player, agent in games[game][mapa].items():
            if players.get(player) != None:
                if (players[player].get(mapa) != None):
                    players[player][mapa].append(agent)
                else:
                    players[player][mapa] = []
            else:
                players[player] = {}
                players[player][mapa] = [agent]
                
# Stats de cada jogador com cada agente

display(Markdown("**Pickrate de cada player:**"))
for player in players.keys():
    display(Markdown(f"**Player: {player}**"))
    for mapa in players[player].keys():
        print("Map:", mapa)
        for agent in set(players[player][mapa]):
            agent_pctg = players[player][mapa].count(agent)/len(players[player][mapa])
            print("Agent: {} - Pick Rate: {:.2f}%".format(agent, agent_pctg*100))
        print("")
    print("")

**Pickrate de mapas:**

Lotus -> 13.64%
Haven -> 18.18%
Pearl -> 22.73%
Icebox -> 13.64%
Ascent -> 18.18%
Split -> 13.64%


**Pickrate de cada player:**

**Player: Shyy**

Map: Lotus
Agent: Killjoy - Pick Rate: 100.00%

Map: Haven
Agent: Killjoy - Pick Rate: 100.00%

Map: Pearl
Agent: Killjoy - Pick Rate: 50.00%
Agent: Skye - Pick Rate: 50.00%

Map: Icebox
Agent: Killjoy - Pick Rate: 100.00%

Map: Ascent
Agent: Killjoy - Pick Rate: 33.33%
Agent: Skye - Pick Rate: 66.67%

Map: Split
Agent: Skye - Pick Rate: 100.00%




...

Map: Lotus
Agent: Sova - Pick Rate: 33.33%
Agent: Skye - Pick Rate: 66.67%

Map: Haven
Agent: Sova - Pick Rate: 100.00%

Map: Pearl
Agent: Skye - Pick Rate: 50.00%
Agent: Raze - Pick Rate: 50.00%

Map: Icebox
Agent: Sova - Pick Rate: 50.00%
Agent: Skye - Pick Rate: 50.00%

Map: Ascent
Agent: Sova - Pick Rate: 33.33%
Agent: Raze - Pick Rate: 66.67%

Map: Split
Agent: Raze - Pick Rate: 100.00%




**Player: Mazino**

Map: Lotus
Agent: Omen - Pick Rate: 33.33%
Agent: Harbor - Pick Rate: 66.67%

Map: Haven
Agent: Omen - Pick Rate: 66.67%
Agent: Harbor - Pick Rate: 33.33%

Map: Pearl
Agent: Astra - Pick Rate: 50.00%
Agent: Harbor - Pick Rate: 50.00%

Map: Icebox
Agent: Harbor - Pick Rate: 100.00%

Map: Ascent
Agent: Omen - Pick Rate: 33.33%
Agent: Astra - Pick Rate: 66.67%

Map: Split
Agent: Astra - Pick Rate: 100.00%




**Player: Tacolilla**

Map: Lotus
Agent: Jett - Pick Rate: 66.67%
Agent: Raze - Pick Rate: 33.33%

Map: Haven
Agent: Jett - Pick Rate: 100.00%

Map: Pearl
Agent: Jett - Pick Rate: 100.00%

Map: Icebox
Agent: Jett - Pick Rate: 100.00%

Map: Ascent
Agent: Jett - Pick Rate: 100.00%

Map: Split
Agent: Jett - Pick Rate: 100.00%




In [34]:
# Análise do arquivo JSON da NRG Esports

with open("nrg-esports.json", "r") as f:
    games = json.load(f)

map_picks = {}
total_picks = 0

for team, maps in games.items():
    for _ in maps.values():
        total_picks += 1

for game in games.keys():
    for mapa in games[game].keys():
        if map_picks.get(mapa) != None:
            map_picks[mapa] += 1
        else:
            map_picks[mapa] = 1
           
display(Markdown("**Pickrate de mapas:**"))
for mapa, num in map_picks.items():
    pctg = round((num/total_picks)*100, 2)
    print(f"{mapa} -> {pctg}%")
    
players = {}
for game in games:
    for mapa in games[game].keys():
        for player, agent in games[game][mapa].items():
            if players.get(player) != None:
                if (players[player].get(mapa) != None):
                    players[player][mapa].append(agent)
                else:
                    players[player][mapa] = []
            else:
                players[player] = {}
                players[player][mapa] = [agent]
                
# Stats de cada jogador com cada agente

display(Markdown("**Pickrate de cada player:**"))
for player in players.keys():
    display(Markdown(f"**Player: {player}**"))
    for mapa in players[player].keys():
        print("Map:", mapa)
        for agent in set(players[player][mapa]):
            agent_pctg = players[player][mapa].count(agent)/len(players[player][mapa])
            print("Agent: {} - Pick Rate: {:.2f}%".format(agent, agent_pctg*100))
        print("")
    print("")

**Pickrate de mapas:**

Bind -> 14.29%
Split -> 14.29%
Haven -> 14.29%
Pearl -> 23.81%
Ascent -> 9.52%
Fracture -> 4.76%
Icebox -> 9.52%
Lotus -> 9.52%


**Pickrate de cada player:**

**Player: crashies**

Map: Bind
Agent: Skye - Pick Rate: 100.00%

Map: Split
Agent: Sova - Pick Rate: 50.00%
Agent: Skye - Pick Rate: 50.00%

Map: Haven
Agent: Sova - Pick Rate: 50.00%
Agent: Skye - Pick Rate: 50.00%

Map: Pearl
Agent: Killjoy - Pick Rate: 25.00%
Agent: Sova - Pick Rate: 25.00%
Agent: Skye - Pick Rate: 50.00%

Map: Ascent
Agent: Sova - Pick Rate: 100.00%

Map: Fracture

Map: Icebox
Agent: Viper - Pick Rate: 100.00%

Map: Lotus
Agent: Killjoy - Pick Rate: 100.00%




...

Map: Bind
Agent: Sage - Pick Rate: 100.00%

Map: Split
Agent: Jett - Pick Rate: 50.00%
Agent: Sage - Pick Rate: 50.00%

Map: Haven
Agent: Jett - Pick Rate: 50.00%
Agent: Sage - Pick Rate: 50.00%

Map: Pearl
Agent: Raze - Pick Rate: 25.00%
Agent: Jett - Pick Rate: 25.00%
Agent: Sage - Pick Rate: 50.00%

Map: Ascent
Agent: Jett - Pick Rate: 100.00%

Map: Fracture

Map: Icebox
Agent: Killjoy - Pick Rate: 100.00%

Map: Lotus
Agent: Raze - Pick Rate: 100.00%




**Player: FiNESSE**

Map: Bind
Agent: Viper - Pick Rate: 100.00%

Map: Split
Agent: Killjoy - Pick Rate: 50.00%
Agent: Viper - Pick Rate: 50.00%

Map: Haven
Agent: Breach - Pick Rate: 50.00%
Agent: Viper - Pick Rate: 50.00%

Map: Pearl
Agent: Killjoy - Pick Rate: 25.00%
Agent: Skye - Pick Rate: 25.00%
Agent: Viper - Pick Rate: 50.00%

Map: Ascent
Agent: Breach - Pick Rate: 100.00%

Map: Fracture

Map: Icebox
Agent: Skye - Pick Rate: 100.00%

Map: Lotus
Agent: Skye - Pick Rate: 100.00%




**Player: Victor**

Map: Bind
Agent: Raze - Pick Rate: 100.00%

Map: Split
Agent: Kayo - Pick Rate: 50.00%
Agent: Raze - Pick Rate: 50.00%

Map: Haven
Agent: Killjoy - Pick Rate: 50.00%
Agent: Raze - Pick Rate: 50.00%

Map: Pearl
Agent: Kayo - Pick Rate: 25.00%
Agent: Viper - Pick Rate: 25.00%
Agent: Raze - Pick Rate: 50.00%

Map: Ascent
Agent: Killjoy - Pick Rate: 100.00%

Map: Fracture

Map: Icebox
Agent: Jett - Pick Rate: 100.00%

Map: Lotus
Agent: Viper - Pick Rate: 100.00%




# Conclusão

Como já dito anteriormente, esse projeto focado na análise de pickrate de mapas e agentes, desempenha um papel fundamental na compreensão das estratégias e preferências dos jogadores e equipes. Ao coletar e analisar dados sobre a frequência com que cada mapa e agente é escolhido, é possível obter insights valiosos sobre as táticas mais populares, a eficácia dos agentes em diferentes situações e as preferências das equipes competitivas.

Em resumo, o uso do Web Scraping para analisar o pickrate de mapas e agentes no Valorant proporciona uma compreensão mais profunda das tendências e preferências dessas equipes. Isso auxilia as equipes no desenvolvimento de estratégias mais eficazes, na identificação de pontos fortes e fracos de cada mapa e agente, e na adaptação às mudanças no meta do jogo. Essa análise de dados é essencial para o scouting de times de Valorant e pode fazer a diferença na busca pela vitória nas competições.

Por fim, a explicação para a "pequena" quantidade de dados é que no campeonato usado para a raspagem de dados (VCT Americas) possui 10 times, sendo 1 deles o time brasileiro para qual eu pensei em fazer esse projeto (LOUD) e 3 desses times seriam os "rivais" do time brasileiro (LOUD): NRG, Leviatán e Cloud9. Por serem apenas 3 times que poderiam trazer algum desconforto para a equipe brasileira campeã mundial, a quantidade de dados acabou não sendo das maiores. 

**Curiosidade:** A equipe brasileira LOUD acabou de ser campeã do VCT Americas, campeonato usado para coletar os dados. Segue o link: https://www.vlr.gg/189055/loud-vs-nrg-esports-champions-tour-2023-americas-league-gf